# Proof of concept - Junction 2020


First install the depedencies and import the needed labriaries

In [1]:
!pip3 install aitoai==0.4.0

In [2]:
import pandas as pd
import numpy as np
from aito.schema import AitoStringType, AitoTextType, AitoDelimiterAnalyzerSchema, AitoTableSchema, AitoColumnLinkSchema, AitoDatabaseSchema
from aito.client import AitoClient
import aito.api as aito_api

Let's define the parameters needed to access the Aito.ai database:

In [3]:
# aito credentials
AITO_INSTANCE_URL = 'https://team1junction.aito.app'
AITO_API_KEY = 'l9uG4dc5ha7BoqBcgT58S2Z7G7uq75ES9w6scxTM'

# define a client to access to the dataset
client = AitoClient(instance_url=AITO_INSTANCE_URL, api_key=AITO_API_KEY)

## Search for places that match an user's criteria

Let's assumme that this user like Italian cuisine

In [14]:
# query definition
query = {
    "from": "places",
    "where" : {
        "cuisine" : {
            "$match" : "Italian" 
        }
    }
}

# send the query
res = aito_api.generic_query(client=client, query=query)
print(res.to_json_string(indent=2))

{
  "offset": 0,
  "total": 28,
  "hits": [
    {
      "address": "Urho Kekkosen Katu 1, Kampin kauppakeskus, 00100, Helsinki",
      "cuisine": "RESTAURANTS & CAFES;Italian",
      "lat": "60.16955184936523",
      "lon": "24.93374443054199",
      "name": "Don Corleone",
      "placeID": 132854,
      "url": "https://doncorleone.fi/"
    },
    {
      "address": "Bulevardi 2-4, 00120, Helsinki",
      "cuisine": "RESTAURANTS & CAFES;Bistro;Italian",
      "lat": "60.1662483215332",
      "lon": "24.94289970397949",
      "name": "Toscanini",
      "placeID": 1469,
      "url": "https://www.toscanini.fi/"
    },
    {
      "address": "Mikonkatu 15, 00100, Helsinki",
      "cuisine": "RESTAURANTS & CAFES;Italian;Pizza",
      "lat": "60.17094039916992",
      "lon": "24.9455623626709",
      "name": "Vapiano",
      "placeID": 135059,
      "url": "http://www.vapiano.fi"
    },
    {
      "address": "Unioninkatu 18, 00130, Helsinki",
      "cuisine": "RESTAURANTS & CAFES;Italian",


10

## We try to predict the top 10 resturant preferences of a group of users

In [23]:
# query
rec_query = {
    "from": "ratings",
    "where": {
        "userID": {
            "$or": [
                {"userID": "U1022"},
                {"userID": "U1026"},
                {"userID": "U1024"},
                {"userID": "U1025"},
            ]
        }
    },
    "recommend": "placeID",
    "goal" : {"rating" : 2},
    "limit": 10,
}

# send query
res = aito_api.recommend(client=client, query=rec_query)
res.json

{'offset': 0,
 'total': 644,
 'hits': [{'$p': 0.8211266897082656,
   'address': 'Annankatu 26, 00100, Helsinki',
   'cuisine': 'RESTAURANTS & CAFES;Asian;Chinese',
   'lat': '60.16724014282226',
   'lon': '24.93623924255371',
   'name': 'Long Wall',
   'placeID': 134986,
   'url': 'https://www.longwall.org/'},
  {'$p': 0.7471362224502227,
   'address': 'Työpajankatu 2a R3, Teurastamo, 00580, Helsinki',
   'cuisine': 'Finnish;RESTAURANTS & CAFES',
   'lat': '60.1900749206543',
   'lon': '24.97002410888672',
   'name': 'Tislaamo - Distillery Bar',
   'placeID': 135034,
   'url': 'https://hdco.fi/tislaamo-distillery-bar/'},
  {'$p': 0.7035453189550154,
   'address': 'Museokatu 46, 00100, Helsinki',
   'cuisine': 'RESTAURANTS & CAFES;Brunch;Italian',
   'lat': '60.17512893676758',
   'lon': '24.919076919555664',
   'name': 'Buongiorno caffe',
   'placeID': 135045,
   'url': 'https://www.facebook.com/buongiorno.kahvila/'},
  {'$p': 0.6855373944942393,
   'address': 'Puustellinpolku 8, 00410

## Retrieve the names of the 10 restaurants

In [26]:
hits = res.json['hits']
names = []
for h in hits:
    names.append(h["name"])
names

['Long Wall',
 'Tislaamo - Distillery Bar',
 'Buongiorno caffe',
 'Ravintola Willihanhi',
 'Kulma - keittö & baari',
 'Mekong restaurant',
 'Vapiano',
 'Puotilan kartano',
 'Restaurant Messenius',
 'The TRAY']